In [1]:
# import the socket library provided by Python
# theoretically, you can only use the socket library to create an Internet server/client
# however, the socket library is a low-level library
# it may not easy to use socket library only to create complicated network-based apps

# if you are interested in all functions provided by Python, see this
# https://docs.python.org/3.7/library/socket.html

import socket

In [2]:
# As a server, we would like to serve multiple clients
# so we use one thread to serve one client
# threading is a python library that can spawn multiple threads in a single app (process)

import threading

In [3]:
# create socket instance
# socket.AF_INET is a constant value that indicates I want to use IP (Internet Protocol) as my L3 protocol
# socket.SOCK_STREAM is a constant value that indicates I want to use TCP as my L4 protocol

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [4]:
# ask the OS to bind the created socket to user-specified parameters: (IP, TCP port)
# the 1st parameter is the binded IP, the 2nd parameter is the binded port number
# So that when the OS receives a datagram, the OS knows how to demux the datagram to the corresponding application 

#TCP port 8889
pars = ('127.0.0.1', 8700) # you can change the server port to whatever you want
s.bind(pars)

In [5]:
# become a server socket
# it makes this python program waiting for receiving message
# listen() function has one parameter that limits how many clients can be connected to this server
# we set it to 5 without any reason
# you can change it to any number you want, as long as you have sufficient resources (computing, memory)

#serves only one client
s.listen(1)

In [6]:
# https://kuanyuchen.gitbooks.io/python3-tutorial/er_jin_zhi_chu_li_fang_shi.html
# http://www.codedata.com.tw/book/java-basic-source/ch4-2.htm

In [7]:
# a new thread is created for every new accepted client
# every new thread starts from the function below
import struct
import binascii
import random

def serveClient(clientsocket, address):
    
    # we need a loop to continuously receive messages from the client
    while True:
        # then receive at most 1024 bytes message and store these bytes in a variable named 'data'
        # you can set the buffer size to any value you like
        data = clientsocket.recv(1024)
      
        pkt1 = struct.unpack('!BBHHHL', data)
        print("from client", pkt1)
        #print("Type: " + binascii.hexlify(pkt1) ) 
        
        # if the received data is not empty, then we send something back by using send() function
        if data:
            Type = 0
            Code =255
            Unused = 0
            Identifier =random.randint(0,2^16)
            seq =1
            msg =106306014
            pkt = struct.pack('!BBHHHL',Type, Code, Unused, Identifier, seq, msg)
            clientsocket.send(pkt)
        
        # we need some condition to terminate the socket
        # lets see if the client sends some termination message to the server
        # if so, then the server close the socket 
        if data == b'close':
            clientsocket.close()
            break

In [ ]:
# since at most we can serve many clients (5 in this example), we need a way to distinguish them 
# as mentioned in the class, TCP use 4-tuple (src IP, dst IP, src port, dst port) to distinguish a socket
# we use accept() function to confirm that we connect to the client socket
# and accept() function will return the client's socket instance and IP
# we need a loop to keep accepting new clients (until 5 clients are accepted)

while True:
    # accept a new client and get it's information
    (clientsocket, address) = s.accept()
    
    # create a new thread to serve this new client
    # after the thread is created, it will start to execute 'target' function with arguments 'args' 
    threading.Thread(target = serveClient, args = (clientsocket, address)).start()


from client (8, 255, 0, 3, 1, 106306014)


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-bce4f0530470>", line 15, in serveClient
    pkt1 = struct.unpack('!BBHHHL', data)
struct.error: unpack requires a buffer of 12 bytes

